# Before Building Model

In [ ]:
from IPython.display import clear_output
!pip install pycaret[full]
clear_output()

import numpy as np
import pandas as pd
import os,random,warnings
warnings.simplefilter('ignore')

from pycaret.classification import *

TRAIN_PATH = "../input/tabular-playground-series-feb-2022/train.csv"
TEST_PATH = "../input/tabular-playground-series-feb-2022/test.csv"
SAMPLE_SUBMISSION_PATH = "../input/tabular-playground-series-feb-2022/sample_submission.csv"
SUBMISSION_PATH = "submission.csv"

ID = "row_id"
TARGET = "target"

MODEL_BLEND_MODELS = ['rf']
MODEL_TRAIN_SIZE = 0.7
MODEL_NORMALIZE = True
MODEL_USE_GPU = True
MODEL_SILENT = True
MODEL_FOLD = 5
MODEL_BLEND_FOLD = 5
MODEL_TUNE_FOLD = 5
MODEL_N_JOBS = -1
MODEL_EVAL_METRIC = 'Accuracy'

SEED = 2022
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything()

# Build Model

In [ ]:
train = pd.read_csv(TRAIN_PATH)

setup(
    data = train,
    target = TARGET,
    train_size = MODEL_TRAIN_SIZE,
    normalize=MODEL_NORMALIZE,
    use_gpu = MODEL_USE_GPU, 
    silent = MODEL_SILENT, 
    fold = MODEL_FOLD, 
    n_jobs = MODEL_N_JOBS
) 

train_models = []
for model in MODEL_BLEND_MODELS:
    trainedModel = create_model(model,fold = MODEL_FOLD)
    train_models.append(trainedModel)

blendModel = blend_models(estimator_list=train_models,optimize=MODEL_EVAL_METRIC,fold = MODEL_BLEND_FOLD)
predict_model(blendModel)

tuneModel = tune_model(blendModel,fold=MODEL_TUNE_FOLD)
predict_model(tuneModel)

final_model = finalize_model(tuneModel)
predict_model(final_model)

# After Building Model

In [ ]:
test = pd.read_csv(TEST_PATH)
predictions=predict_model(final_model,data=test)

sub = pd.read_csv(SAMPLE_SUBMISSION_PATH)
sub[TARGET] = predictions.Label
sub.to_csv(SUBMISSION_PATH,index=False)
sub.head()